In [1]:
from keras.models import Model 
from keras.layers import Input, BatchNormalization, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Conv1D, UpSampling1D, Conv1DTranspose, Reshape
import np_utils
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
import os
from sklearn.preprocessing import StandardScaler
import os
import datetime
from sklearn.model_selection import train_test_split

In [2]:
poor = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_poor.csv", header=None)
middle = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_middle.csv", header=None)
rich = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_rich.csv", header=None)
sc = StandardScaler()
bank_reserves = []
for i in np.arange(0, poor.shape[0]):
    sample = pd.concat([poor.iloc[i], middle.iloc[i]], axis=0).T
    sample = pd.concat([sample, rich.iloc[i]], axis=0).T
    sample_std = sc.fit_transform(sample.to_frame())
    bank_reserves.append(sample_std)

In [3]:
bank_reserves = [sample.flatten() for sample in bank_reserves]
bank_reserves = np.asarray(bank_reserves)
bank_reserves_train, bank_reserves_test = train_test_split(bank_reserves, test_size=0.20, random_state=42)

In [4]:
bank_reserves_train = bank_reserves_train.reshape(bank_reserves_train.shape[0], 303, 1)
bank_reserves_test = bank_reserves_test.reshape(bank_reserves_test.shape[0], 303, 1)

In [5]:
bank_reserves.shape

(100000, 303)

In [6]:
input = Input(shape=(303,1))

encoded = Conv1D(filters=32, kernel_size=10, activation='relu', padding='same')(input)
encoded = BatchNormalization()(encoded)
encoded = Conv1D(filters=64, kernel_size=10, activation='relu', padding='same')(encoded)
encoded = BatchNormalization()(encoded)
encoded = Conv1D(filters=128, kernel_size=10, activation='relu', padding='same')(encoded)
encoded = BatchNormalization()(encoded)
encoded = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(encoded)
encoded = BatchNormalization()(encoded)

encoded = Flatten()(encoded)

encoded = Dense(303, activation='relu')(encoded)
decoded = Dense(5, activation='relu')(encoded)
decoded = Dense(303, activation='relu')(decoded)

decoded = Reshape((303, 1))(decoded)

decoded = Conv1DTranspose(filters=128, kernel_size=3, activation='relu', padding='same')(decoded)
decoded = BatchNormalization()(decoded)
decoded = Conv1DTranspose(filters=128, kernel_size=10, activation='relu', padding='same')(decoded)
decoded = BatchNormalization()(decoded)
decoded = Conv1DTranspose(filters=64, kernel_size=10, activation='relu', padding='same')(decoded)
decoded = BatchNormalization()(decoded)
decoded = Conv1DTranspose(filters=32, kernel_size=10, activation='relu', padding='same')(decoded)
decoded = BatchNormalization()(decoded)
decoded = Conv1DTranspose(filters=1, kernel_size=10, activation=None, padding='same')(decoded)

autoencoder = Model(input, decoded)
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 303, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 303, 32)        │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 303, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 303, 64)        │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 303, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 303, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 303, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 303, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 303, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 38784)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 303)            │    11,751,855 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 303)            │         1,818 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 303, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose                │ (None, 303, 128)       │           512 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 303, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_1              │ (None, 303, 128)       │       163,968 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 303, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_2              │ (None, 303, 64)        │        81,984 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 303, 64)        │           256 │
│ (BatchNormalization)            │                        │             

 Total params: 12,177,530 (46.45 MB)

 Trainable params: 12,176,122 (46.45 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [7]:
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [8]:
autoencoder.fit(bank_reserves_train, bank_reserves_train,
 epochs=20,
 batch_size=64,
 shuffle=True,
 validation_data=(bank_reserves_test, bank_reserves_test),
 verbose=1)

Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 511s 405ms/step - loss: 0.0905 - val_loss: 0.0066
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 496s 397ms/step - loss: 0.0057 - val_loss: 0.0076
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 502s 401ms/step - loss: 0.0055 - val_loss: 0.0062
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 496s 396ms/step - loss: 0.0054 - val_loss: 0.0060
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 492s 393ms/step - loss: 0.0053 - val_loss: 0.0060
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 508s 406ms/step - loss: 0.0053 - val_loss: 0.0064
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 495s 396ms/step - loss: 0.0052 - val_loss: 0.0058
Epoch 8/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 498s 398ms/step - loss: 0.0052 - val_loss: 0.0082
Epoch 9/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 497s 398ms/step - loss: 0.0051 - val_loss: 0.0067
Epoch 10/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 490s 392ms/step - loss: 0.0051 - val_loss: 0.0076
Epoch 11/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 487s 390ms/step - loss: 0.0050 - val

In [9]:
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(index=6).output)
encoded_ts = encoder.predict(bank_reserves)

625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step


In [10]:
encoded_ts[0].shape

(303, 128)